# 뉴스 데이터 합치기

In [1]:
# 날짜 형식 datetime으로 바꾸기

In [21]:
import pandas as pd
import datetime

In [22]:
df1 = pd.read_csv('연합뉴스_전처리(1).csv')

t = df1['time']
df1['time'] = [datetime.datetime.strptime(i, '%Y-%m-%d') for i in t]

In [23]:
df2 = pd.read_csv('연합인포맥스_전처리(1).csv')

r = df2['time']
df2['time'] = [datetime.datetime.strptime(i, '%Y-%m-%d') for i in r]

In [31]:
df3 = pd.read_csv('이데일리_전처리.csv')

e = df3['time']
df3['time'] = [i.strip()[:-1].replace('.', '-') for i in e]
df3['time'] = pd.to_datetime(df3['time'], format='%Y-%m-%d', errors='raise')
ind_del = df3[df3['time'] == ''].index
df3 = df3.drop(ind_del)

In [ ]:
# 3사 뉴스 합치기

In [33]:
df4 = pd.concat([df1,df2])
df = pd.concat([df3, df4])
df = df[['time', 'body']]
df

,time,body
0,2005-07-01,미국 연방준비제도이사회 FRB 가 연방기금금리 금리 목표치를 5%로 25bp 5%p...
1,2005-07-01,30일 유럽증시 주요 지수들이 보합권에서 혼조세를 보이며 마감했다. 영국과 독일이 ...
2,2008-03-07,한국은행이 7일 금융통화위원회를 열어 기준금리를 5%로 결정했다. 이는 기존 콜금리...
3,2008-02-14,14일 현지시간 뉴욕 주식시장이 개장 전 거래에서 상승세를 타고 있다. 주간 신...
4,2008-01-31,31일 채권 금리가 사흘만에 하락했다. 미국이 예상대로 50bp의 기준금리 인하를...
...,...,...
59815,2017-03-02,미국 연방준비제도 연준 Fed 내에서 강경 비둘기파로 분류되는 라엘 브레이너드 이...
59816,2017-03-02,국채선물이 3월 미국 기준금리 인상 우려가 확대됨에 따라 장기물을 중심으로 급락 출...
59817,2017-03-02,국제경제부 1일 미국시간 뉴욕 금융시장에서 주요 주가지수들은 도널드 트럼프 미국...
59818,2017-03-01,달러엔은 도널드 트럼프 미국 대통령의 의회 연설이 시작되자 차익매물이 나오며 상승폭...


In [94]:
df = df.groupby('time').sum()
df.to_csv('news_pre.csv')
df.to_pickle('news_pre.pkl')

In [ ]:
# 의사록 + 뉴스 + 채권분석리포트 합치기

In [65]:
bond = pd.read_csv('bond_data.csv')

In [66]:
# 날짜 형식 맞추기
e = bond['date']

time_list = []
for i in e:
    y, m, d = i.split('.')
    full_y = '20'+y
    time = full_y + '-'+ m + '-' + d
    time_list.append(time)
bond['date'] = time

bond['date'] = pd.to_datetime(bond['date'], format='%Y-%m-%d', errors='raise')

In [68]:
bond.rename(columns={"date": "time", "text": "body"}, inplace = True)
bond = bond[['time', 'body']]

In [69]:
mpb = pd.read_pickle('mpb_all.pkl')
mpb['date'] = pd.to_datetime(mpb['date'], format='%Y-%m-%d', errors='raise')
mpb.rename(columns={"date": "time", "body": "body"}, inplace = True)
mpb = mpb[['time', 'body']]

In [71]:
mpb_bond = pd.concat([mpb,bond])
concat = pd.concat([df, mpb_bond])
concat = concat[['time', 'body']]
concat

,time,body
0,2005-07-01,미국 연방준비제도이사회 FRB 가 연방기금금리 금리 목표치를 5%로 25bp 5%p...
1,2005-07-01,30일 유럽증시 주요 지수들이 보합권에서 혼조세를 보이며 마감했다. 영국과 독일이 ...
2,2008-03-07,한국은행이 7일 금융통화위원회를 열어 기준금리를 5%로 결정했다. 이는 기존 콜금리...
3,2008-02-14,14일 현지시간 뉴욕 주식시장이 개장 전 거래에서 상승세를 타고 있다. 주간 신...
4,2008-01-31,31일 채권 금리가 사흘만에 하락했다. 미국이 예상대로 50bp의 기준금리 인하를...
...,...,...
2775,2008-04-01,Microsoft Word - FixedIncome_0428.doc ...
2776,2008-04-01,Microsoft Word - Fixedincome0414.doc ...
2777,2008-04-01,Microsoft Word - 0407.doc Fixed Income Weekl...
2778,2008-04-01,Microsoft Word - 0407.doc Fixed Income Weekl...


In [72]:
concat = concat.groupby('time').sum()

In [77]:
concat.to_pickle('concat.pkl')

# Concat 파일 불러와서 n-gram

In [74]:
import pandas as pd
df = pd.read_pickle('concat.pkl')

In [6]:
import pandas as pd
from tqdm import tqdm
from ekonlpy.sentiment import MPCK
mpck = MPCK()
# 기사 문장별 split 후 ngramize
ngram_list=[]
for i in tqdm(range(len(df['body']))):
    try:
        sent_ngram=[]
        sent=df['body'][i].split('.')
        for s in sent:
            ngrams = mpck.ngramize(mpck.tokenize(s),keep_overlapping_ngram=False)
            if ngrams:
                sent_ngram.append(ngrams)
        ngram_list.append(sent_ngram)
    except:
        print(i)
        
df['ngram']=ngram_list

100%|████████████████████████████████████████████████████████████████████████████| 4611/4611 [1:31:57<00:00,  1.20s/it]
C:\Users\hyeryeon\.conda\envs\python36\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
df['ngram']=ngram_list

In [8]:
df.to_csv('concat_ngram.csv')
df.to_pickle('concat_ngram.pkl')